### 1. Создайте список из 1000 случайных строк, состоящих из латинских букв.

In [1]:
import random
import string
import numpy as np

In [2]:
def generate_random_str_list(min_len, max_len, count=1000, char_set=string.ascii_letters):
    if min_len >= max_len:
        raise ValueError("bad min/max values")
    
    return [''.join(random.choice(char_set) for i in range(min_len, random.randint(min_len + 1, max_len))) \
            for i in range(count)]

In [3]:
rand_strs = generate_random_str_list(1, 100)

### 2. Каждая строка - случайной длиной от 3 до 8 символов.

In [4]:
rand_strs = generate_random_str_list(3, 8)

### 3. Реализуйте бинарное дерево для строк
Каждый узел дерева будет списком (кортежи не подойдут из-за неизменяемости).
Первый элемент списка - значение узла, второй - левый узел (список или None),
третий - правый узел (список или None).

In [5]:
def tree_insert(root, s):
    if len(root) == 0:
        root.extend([s, None, None])
        return

    parent = None
    node = root
    
    while node != None:
        value, left, right = node[0], node[1], node[2]
        parent = node
        
        if s < value:
            node = left;
        else:
            node = right

    value = parent[0]
    new_node = [s, None, None]
    if s < value:
        parent[1] = new_node
    else:
        parent[2] = new_node

Построение дерева будет работать для любых типов, допускающих сравнение. Тестирование на числовых данных:

In [6]:
# Result:
#                           100
#                          /   \
#                        50     150
#                       /  \
#                     49    51

root = []

tree_insert(root, 100)
tree_insert(root, 50)
tree_insert(root, 49)
tree_insert(root, 150)
tree_insert(root, 51)

print(root)

[100, [50, [49, None, None], [51, None, None]], [150, None, None]]


Вставка ранее сгенерированных строк:

In [7]:
root = []
for s in rand_strs:
    tree_insert(root, s)

### 4. Реализуйте поиск ближайшего слова для заданного.

В качестве метрики будем использовать расстояние Левенштейна
$$d(S_1, S_2) = D(M, N),$$
где

$$D(i, j) = 
\begin{cases}
    0, & i = 0, j = 0 \\
    i * c_2, & j = 0, i > 0 \\
    j * c_1, & i = 0, j > 0 \\
    \min(D(i - 1, j) + c_1, D(i, j - 1) + c_2, D(i - 1, j - 1) + c_3), & i > 0, j > 0 \\
 \end{cases}$$
 
 $c_1$, $c_2$ и $c_3$ - стоимость вставки, удаления и замены одного символа
 соответственно.

In [62]:
def levenstein_dist(s1, s2, insert_cost, delete_cost, replace_cost):
    m = len(s1)
    n = len(s2)
    d = np.zeros((m + 1, n + 1), dtype=np.int)
    
    for j in range(1, n + 1):
        d[0][j] = d[0][j - 1] + insert_cost
    
    for i in range(1, m + 1):
        d[i][0] = d[i - 1][0] + delete_cost
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] != s2[j - 1]:
                d[i][j] = min(d[i - 1][j] + delete_cost,
                              d[i][j - 1] + insert_cost,
                              d[i - 1][j - 1] + replace_cost)
            else:
                d[i][j] = d[i - 1][j - 1]
    
    return d[m][n]

Стоимости вставки, удаления и замены сделаем одинаковыми.

In [67]:
insert_cost = 1
delete_cost = 1
replace_cost = 1

levenstein_dist("connect", "conehead", insert_cost, delete_cost, replace_cost)

4

In [68]:
s = rand_strs[random.randint(0, len(rand_strs))]
ht = {}

for item in rand_strs:
    dist = levenstein_dist(s, item, insert_cost, delete_cost, replace_cost)
    words = ht.get(dist)
    
    if words == None:
        ht[dist] = [item]
    else:
        words.append(item)
        ht[dist] = words

dist_words_pairs = list(zip(ht.keys(), ht.values()))
dist_words_pairs.sort()

print(s)

for dist, words in dist_words_pairs[:10]:
    print(dist, words[:3])

VGEtD
0 ['VGEtD']
3 ['yGnD', 'MmtD', 'VZYTD']
4 ['GLw', 'YEY', 'DEdb']
5 ['d', 'YCIke', 'foC']
